In [2]:
import datetime, os, pickle
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
#functions from LSTM script commented '#LSTM'
from Scripts.LSTM import *

%matplotlib inline
plt.style.use('seaborn-whitegrid')

models_path = os.path.join('..', 'models')
images_path = os.path.join('..', 'images')
results_path = os.path.join('..', 'results')
data_path = os.path.join('..', 'btc_data')

Using TensorFlow backend.


In [ ]:
os.system("start C:/Users/Giedrius/Music/1.mp3") 

# Long Short-Term Memory (LSTM) Recurrent Neural Network



Long Short-Term Memory ([LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)) networks are a type of recurrent neural network capable of learning overlong sequences. LSTM achieves state of the art performance on important tasks that include language modeling, speech recognition, and machine translation.

LSTM has advantages on time series forecasting over classical methods like ARIMA because itself can learn time series trend, seasonality and perform well on non-stationarity data. We'll try to use these model properties to predict Bitcoin price.

But before getting further into Bitcoin price modeling lets look how powerful LSTM is in approximating continues function e.g.  

$$f(x)=\frac{x\cdot\sin(x)}{400}$$

In [ ]:
#define x and f(x)
x = np.arange(0.0, 100, 0.1)
ts =  x*np.sin(x) / 400
#plot f(x)
plt.figure(figsize=(15,4))
plt.plot(x,ts, label = "f(x) = x * sin(x)/400")
plt.legend(prop={'size': 14})

We'll prepare  $f(x)=\frac{x\cdot\sin(x)}{400}$ data for LSTM; we will reshape time series array into numpy array of 3 dimensions (N, W, F) where N is the number of training sequences, W is the sequence length (*window_size* in the following code) and F is the number of features of each sequence (in univariate case is equal 1). The sequences themselves are sliding) windows and hence shift by 1 each time, causing a constant overlap with the prior windows.

In [ ]:
window_size = 50
x_train, y_train, x_test, y_test, rows = transform_data(ts, window_size, normalise_window = False) #LSTM

x_a = np.reshape(x_train,[855, 50] )
print("First four training samples:")
for i in range(4):
    print("  x[%d]: [" % (i), end= "")
    for j in [0,1,2,47,48,49]:
      print(round(x_a[i][j],5),end= " ")
      if j == 2:
          print("...",end= " ")
    print("]  ", "y[%d]: " % (i), round(y_train[i],5))

We will use a 2 layer LSTM model. The model uses the AdamOptimizer as its optimization function.


In [ ]:
if 1 == 0:
    model = build_model([1,128,128,1], dropout = [0,0]) #LSTM
    loss = model.fit(
        x_train,
        y_train,
        batch_size = 64,
        nb_epoch = 100,
        validation_split = 0.2)
    #model.save(os.path.join(models_path, "xsinx"))
else:
    model = load_network(os.path.join(models_path, "xsinx")) #LSTM

In [ ]:
#create variable for ploting
predictions = predict_sequence_full(model, x_test, window_size, 95) #LSTM
none = [None]*(len(y_train)+ window_size)
ts1 = []
ts1.extend(none)
ts1.extend(predictions)
ts2 = []
ts2.extend(ts[:-(len(y_test)-1)])
ts2.extend([None] * (len(y_test)-1))
#plot
plt.figure(figsize=(15,4))
plt.plot(x, ts2, color='blue', label = 'f(x) = x * sin(x) / 400')
plt.plot(x, ts1, color='orange', label= 'LSTM predictions')
plt.legend(prop={'size': 12})

From plot above we can clearly see that model learn function pattern perfectly.

# Univariate LSTM bitcoin modeling


We will predict Bitcoin opening prices which we'll retrieve from [www.coinmarketcap.com](http://coinmarketcap.com/currencies/bitcoin/historical-data/)

In [ ]:
#get data 
if 1 == 0:
    urlMarket = 'http://coinmarketcap.com/currencies/bitcoin/historical-data/?start=20131227&end=20171201'
    df2 = pd.read_html(urlMarket)[0]
    df2.index  = [datetime.datetime.strptime(date, '%b %d, %Y') for date in df2["Date"]]
    df2.index.name = 'Date'
    df2 = df2[['Open','Volume']].sort_index()
    df = df2['Open'].sort_index()
    df.to_pickle(os.path.join(data_path, 'btc_price'))
else:
    df = pd.read_pickle(os.path.join(data_path, 'btc_price'))
df.plot(figsize = (15,5), title = "Bitcoin opening price")

Bitcoin price is extremely volatile in order to generalize its dynamics and let the model converge to optimums we'll normalize each window separately. We’ll use the following equation for normalization.
$$n_i=\frac{w_i}{w_0}-1$$ 
<p>n - normalised list [window] of BTC price changes <br />
w - raw list [window] of BTC opening prices
<p>The first 90% of the data we will use for data training (which 20% of them splitted in validation data during training) and 10 % for testing.<br />

In [ ]:
window_size = 50
x_train, y_train, x_test, y_test, rows = transform_data(df, window_size, True)
y_test_usd = get_real_value(y_test, df,rows)

We will train a 2 layers LSTM neural network with various parameters combinations to forecast next day Bitcoins price, also for curiosity's sake, we will predict next nine days based on previous predictions e.g. for the sixth-day price prediction previous five days predictions will be used.
Firstly let's tune neurons number in layers, we will try different combinations of  16, 32, 64, 128 and 256 neurons in each layer e.g. (16,16), (16,32) ... Every neurons configuration we will repeat 3 times. The starting model parameters:

* window size 50  
* batches 64 
* dropout 0.2 in both layers
* epochs 250

The model accuracy is evaluated by using two measurements: Mean absolute percentage error (MAPE) and root-mean-square error (RMSE).


%%time

pred_len = 10
repeats = 3
i = 0
neurons = [16,32,64,128,256]
result = []

for n1 in neurons:
    for n2 in neurons:

        rmse_scores, mape_scores = list(), list()
        loss_train, loss_validate = list(), list()
        
        for r in range(repeats):
        
            # fit the model
            model = build_model([1,n1,n2,1])#LSTM
            loss = model.fit(
                x_train,
                y_train,
                batch_size=64,
                nb_epoch=250,
                validation_split=0.2)
        
            # forecast
            predictions = predict_sequences_multiple(model, x_test, window_size, pred_len, True, True df, rows) #LSTM
            predictions_usd = get_seq_real_values(predictions, df, rows) #LSTM
            rmse, mape = calculate_error(predictions_usd, np.array(y_test_usd), pred_len) #LSTM
            rmse_scores.append(rmse)
            mape_scores.append(mape)
            loss_train.append(loss.history["loss"])
            loss_validate.append(loss.history["val_loss"])
            print('%d_%d %d) RMSE: %.3f   MAPE: %.2f' % (n1,n2,r+1, rmse[0], mape[0]))
    
    
        result.append((rmse_scores, mape_scores, loss_train, loss_validate))
        
os.system("start C:/Users/Giedrius/Music/1.mp3")         
with open(os.path.join(results_path, 'neurons'), 'wb') as f:
    pickle.dump(result, f, pickle.HIGHEST_PROTOCOL)

In [3]:
with open(os.path.join(results_path, 'neurons'), "rb") as f:
    result = pickle.load(f)
neurons = [16,32,64,128,256]

neurons_labels = [str(n1)+"_"+ str(n2) for n1 in neurons for n2 in neurons]
rmse = [np.mean(r[0], axis = 0) for r in result]
mape = [np.mean(r[1], axis = 0) for r in result]

mape = pd.DataFrame(mape, index = neurons_labels).transpose()
rmse = pd.DataFrame(rmse, index = neurons_labels).transpose()
mape.index = rmse.index = range(1,11)
mape.index.name = rmse.index.name =  "Prediction"

In [4]:
rmse.style.apply(highlight_min, axis=1)

,16_16,16_32,16_64,16_128,16_256,32_16,32_32,32_64,32_128,32_256,64_16,64_32,64_64,64_128,64_256,128_16,128_32,128_64,128_128,128_256,256_16,256_32,256_64,256_128,256_256
Prediction,,,,,,,,,,,,,,,,,,,,,,,,,
1,432.248,425.209,396.279,644.354,714.209,407.418,325.37,450.387,545.808,618.975,335.087,287.053,319.39,434.865,491.457,348.166,301.766,295.081,312.346,343.337,320.855,298.898,289.486,298.039,327.668
2,619.856,662.216,633.17,931.056,1005.91,621.928,528.189,753.354,864.548,961.29,538.931,466.644,540.708,737.893,799.399,583.982,501.988,489.5,525.614,574.513,534.926,500.229,479.197,494.87,550.495
3,815.526,867.966,823.98,1092.96,1119.7,802.875,684.703,937.865,1007.69,1092.12,698.732,603.764,700.591,918.982,946.093,756.576,655.157,637.208,685.42,730.715,697.64,666.766,633.539,649.348,715.34
4,986.159,1031.17,975.903,1190.6,1179.25,955.332,811.418,1079.46,1101.94,1172,834.22,723.2,835.617,1062.92,1050.28,909.699,790.998,762.358,816.786,853.716,841.284,814.734,763.215,774.641,845.09
5,1124.41,1156.49,1091.79,1248.81,1211.68,1077.4,907.393,1184.72,1155.11,1217.68,940.802,816.589,942.264,1174.07,1120.01,1035.54,899.198,862.063,920.332,950.569,959.54,938.953,866.922,872.349,947.944
6,1236.19,1253.64,1182.36,1282.52,1229.18,1176.6,981.091,1264.87,1182.92,1245.45,1025.77,890.637,1027.09,1261.83,1168.27,1139.45,985.853,942.433,1003.12,1029.47,1054.44,1042.53,950.241,949.542,1031.75
7,1326.69,1329.62,1254.38,1300.95,1238.63,1258.02,1038.38,1326.87,1194.68,1264.07,1094.15,949.889,1094.9,1331.79,1202.4,1225.26,1055.64,1007.66,1069.12,1094.57,1128.78,1127.52,1016.87,1010.39,1099.94
8,1403.96,1392.94,1316.57,1312.28,1246.09,1329.01,1086.57,1379.12,1198.7,1281.59,1153.07,1001.94,1153.51,1392.05,1231.76,1299.82,1116.44,1065.71,1127.05,1154.6,1189.76,1201.61,1074.89,1063.11,1161.31
9,1470.1,1446.14,1370.53,1319.11,1252.7,1390.97,1127.35,1423.38,1198.19,1299.16,1204.25,1047.6,1204.56,1444.25,1257.28,1365.69,1170.22,1117.85,1177.77,1209.35,1240.73,1266.91,1126.06,1109.03,1216.02


In [5]:
mape.style.apply(highlight_min, axis=1)

,16_16,16_32,16_64,16_128,16_256,32_16,32_32,32_64,32_128,32_256,64_16,64_32,64_64,64_128,64_256,128_16,128_32,128_64,128_128,128_256,256_16,256_32,256_64,256_128,256_256
Prediction,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.0609863,0.0579573,0.0532235,0.0746175,0.0803321,0.0554676,0.0455136,0.0580401,0.0651975,0.0723437,0.0466449,0.0419456,0.0452895,0.0563454,0.0603145,0.0489149,0.0443197,0.0428222,0.0442257,0.0475623,0.045954,0.0436555,0.0426007,0.0427667,0.0461
2,0.0842921,0.0865623,0.0813693,0.107054,0.113818,0.0815255,0.0699873,0.0930357,0.0995789,0.11141,0.0717308,0.0651685,0.0728532,0.0915744,0.095234,0.0786437,0.0702148,0.0678378,0.0707314,0.0767728,0.072697,0.0701344,0.0662864,0.0669827,0.0738782
3,0.108362,0.111806,0.103947,0.126481,0.129265,0.10387,0.0887358,0.115327,0.116294,0.129572,0.0911629,0.0822098,0.0925101,0.113721,0.113564,0.100255,0.0897144,0.0857418,0.0899069,0.0967412,0.0926219,0.0910346,0.0853205,0.0850698,0.094173
4,0.129907,0.132226,0.122231,0.140629,0.139916,0.122733,0.104137,0.133661,0.129807,0.143024,0.107698,0.0973898,0.109686,0.132253,0.12826,0.119475,0.107425,0.101748,0.106423,0.113362,0.110055,0.109903,0.101043,0.0998478,0.110529
5,0.14895,0.149209,0.137664,0.1508,0.147946,0.138935,0.116326,0.148545,0.139819,0.153056,0.121001,0.109748,0.123796,0.1478,0.13946,0.136282,0.122364,0.11463,0.120011,0.127493,0.124892,0.126463,0.114682,0.111812,0.124901
6,0.166654,0.164455,0.150618,0.158319,0.153737,0.154119,0.125931,0.160782,0.145352,0.16142,0.132693,0.120567,0.136149,0.160985,0.148366,0.152127,0.135941,0.126341,0.131806,0.141423,0.137948,0.142069,0.126103,0.121187,0.138044
7,0.181537,0.17653,0.161235,0.162636,0.156065,0.166965,0.13334,0.170562,0.146723,0.167285,0.141996,0.129098,0.146263,0.172403,0.153894,0.164999,0.146527,0.136002,0.141806,0.15209,0.147843,0.154939,0.135051,0.127735,0.149102
8,0.195068,0.188292,0.172788,0.16723,0.159765,0.179598,0.142174,0.180597,0.14815,0.173867,0.15256,0.138754,0.157407,0.183549,0.16081,0.177652,0.157468,0.147123,0.15235,0.163723,0.158471,0.168055,0.144894,0.136492,0.160009
9,0.206634,0.198035,0.182323,0.171095,0.1636,0.190362,0.149611,0.189626,0.150512,0.180061,0.161426,0.147262,0.166789,0.193266,0.167934,0.188407,0.166972,0.156251,0.16193,0.174683,0.167155,0.179437,0.153845,0.144578,0.170244


From tables above we see that smallest MAPE and RMSE is in LSTM with 64 neurons in first layer and 32 neurons in second layer.